In [ ]:
!pip install transformers==4.27.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.2 MB/s eta 0:00:00


In [ ]:
from transformers import BertForSequenceClassification, AdamW
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import math
from typing import Optional, Dict, List, Tuple

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("amazon_reviews_multi", "en")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/200000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset amazon_reviews_multi downloaded and prepared to /root/.cache/huggingface/datasets/amazon_reviews_multi/en/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(len(dataset['train']))
print(len(dataset['validation']))
print(len(dataset['test']))

200000
5000
5000


In [ ]:
from datasets import Dataset
import pandas as pd

def switch_label(example):
    if (example['stars'] == 1 or example['stars'] == 2):
      example['stars'] = 0

    elif (example['stars'] == 4 or example["stars"] == 5):
      example['stars'] = 1

    return example

def process_dataset(dataset, file_path: str):
  data = dataset[file_path]
  data = data.filter(lambda x: x['stars'] in [1, 2, 4, 5])
  data = data.map(lambda x: {'title_and_body': x['review_title'] + ' ' + x['review_body']})
  data = data.map(switch_label)
  data = data.to_pandas()

  return data

train_set = process_dataset(dataset, 'train')
val_set = process_dataset(dataset, 'validation')
test_set = process_dataset(dataset, 'test')

In [ ]:
import torch
import numpy as np
import random

def set_random_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# Set the random seed value
seed_value = 42
set_random_seed(seed_value)

In [ ]:
from sklearn.model_selection import train_test_split

train_set, _ = train_test_split(train_set, test_size=0.9, random_state=42)
val_set, _ = train_test_split(val_set, test_size=0.9, random_state=42)
test_set, _ = train_test_split(test_set, test_size=0.9, random_state=42)

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize all sentences and map the tokens to their word IDs
train_encodings = tokenizer(train_set['title_and_body'].tolist(), padding=True, truncation=True)
val_encodings = tokenizer(val_set['title_and_body'].tolist(), padding=True, truncation=True)
test_encodings = tokenizer(test_set['title_and_body'].tolist(), padding=True, truncation=True)

train_labels = torch.tensor(train_set['stars'].tolist(), device=device)
val_labels = torch.tensor(val_set['stars'].tolist(), device=device)
test_labels = torch.tensor(test_set['stars'].tolist(), device=device)

# Define batch size and create DataLoader for each set
batch_size = 4

# Define data loaders
train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids'], device=device), torch.tensor(train_encodings['attention_mask'], device=device), train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = torch.utils.data.TensorDataset(torch.tensor(val_encodings['input_ids'], device=device), torch.tensor(val_encodings['attention_mask'], device=device), val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids'], device=device), torch.tensor(test_encodings['attention_mask'], device=device), test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Initialize BERT model and optimizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

cuda


In [ ]:
# Train the model
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [ ]:
# Evaluate the model on the validation set
model.eval()
total_correct = 0
total_samples = 0
for batch in val_loader:
    with torch.no_grad():
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)
        total_correct += torch.sum(predicted_labels == labels).item()
        total_samples += labels.size(0)
val_acc = total_correct / total_samples
print(f"Validation Accuracy: {val_acc}")

# Evaluate the model on the test set
model.eval()
total_correct = 0
total_samples = 0
for batch in test_loader:
    with torch.no_grad():
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)
        total_correct += torch.sum(predicted_labels == labels).item()
        total_samples += labels.size(0)
test_acc = total_correct / total_samples
print(f"Test Accuracy: {test_acc}")

Validation Accuracy: 0.9175
Test Accuracy: 0.945


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.save_pretrained('BERT_Amazon')
tokenizer.save_pretrained('BERT_Amazon')

('BERT_Amazon/tokenizer_config.json',
 'BERT_Amazon/special_tokens_map.json',
 'BERT_Amazon/vocab.txt',
 'BERT_Amazon/added_tokens.json')

In [ ]:
model.push_to_hub('tsaland/BERT_Amazon')
tokenizer.push_to_hub('tsaland/BERT_Amazon')

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/tsaland/BERT_Amazon/commit/8ed526f7447ef584aeb6c6b9cd55a6e44ef87a65', commit_message='Upload tokenizer', commit_description='', oid='8ed526f7447ef584aeb6c6b9cd55a6e44ef87a65', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model = BertForSequenceClassification.from_pretrained('tsaland/BERT_Amazon', num_labels=2).to(device)
tokenizer = BertTokenizer.from_pretrained('tsaland/BERT_Amazon')

In [ ]:
def test_trigger(model, trigger, test_set):
  test_set['trigger_text'] = trigger + test_set['title_and_body']
  trigger_test_encodings = tokenizer(test_set['trigger_text'].tolist(), padding=True, truncation=True)
  test_labels = torch.tensor(test_set['stars'].tolist(), device=device)
  trigger_test_dataset = torch.utils.data.TensorDataset(torch.tensor(trigger_test_encodings['input_ids'], device=device), torch.tensor(trigger_test_encodings['attention_mask'], device=device), test_labels)
  trigger_test_loader = DataLoader(trigger_test_dataset, batch_size=batch_size)

  # Evaluate model with triggers
  model.eval()
  total_correct = 0
  total_samples = 0
  for batch in trigger_test_loader:
      with torch.no_grad():
          input_ids = batch[0]
          attention_mask = batch[1]
          labels = batch[2]
          outputs = model(input_ids, attention_mask=attention_mask)
          _, predicted_labels = torch.max(outputs.logits, dim=1)
          total_correct += torch.sum(predicted_labels == labels).item()
          total_samples += labels.size(0)
  test_acc = total_correct / total_samples
  print(f"Test Accuracy with triggers {trigger}: {test_acc}")


In [ ]:
import torch.nn.functional as F

def trigger_avg_probs(model, trigger, test_set):
    test_set['trigger_text'] = trigger + test_set['title_and_body']
    trigger_test_encodings = tokenizer(test_set['trigger_text'].tolist(), padding=True)
    test_labels = torch.tensor(test_set['stars'].tolist(), device=device)
    trigger_test_dataset = torch.utils.data.TensorDataset(torch.tensor(trigger_test_encodings['input_ids'], device=device), torch.tensor(trigger_test_encodings['attention_mask'], device=device), test_labels)
    trigger_test_loader = DataLoader(trigger_test_dataset, batch_size=batch_size)

    # Evaluate model with triggers
    model.eval()
    total_samples = 0
    probs_sum = torch.zeros((1, model.num_labels), device=device)
    for batch in trigger_test_loader:
        with torch.no_grad():
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]
            outputs = model(input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)
            probs_sum += torch.sum(probs, dim=0)
            total_samples += labels.size(0)

    avg_probs = probs_sum / total_samples
    return avg_probs

In [ ]:
def test_model(model, test_set):
  test_encodings = tokenizer(test_set['title_and_body'].tolist(), padding=True, truncation=True)
  test_labels = torch.tensor(test_set['stars'].tolist(), device=device)
  test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids'], device=device), torch.tensor(test_encodings['attention_mask'], device=device), test_labels)
  test_loader = DataLoader(test_dataset, batch_size=batch_size)

  # Evaluate model with triggers
  model.eval()
  total_correct = 0
  total_samples = 0
  for batch in test_loader:
      with torch.no_grad():
          input_ids = batch[0]
          attention_mask = batch[1]
          labels = batch[2]
          outputs = model(input_ids, attention_mask=attention_mask)
          _, predicted_labels = torch.max(outputs.logits, dim=1)
          total_correct += torch.sum(predicted_labels == labels).item()
          total_samples += labels.size(0)
  test_acc = total_correct / total_samples
  print(f"Test Accuracy without triggers {test_acc}")


In [ ]:
import torch.nn.functional as F

def avg_probs(model, test_set):
    test_encodings = tokenizer(test_set['title_and_body'].tolist(), padding=True)
    test_labels = torch.tensor(test_set['stars'].tolist(), device=device)
    test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids'], device=device), torch.tensor(test_encodings['attention_mask'], device=device), test_labels)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Evaluate model with triggers
    model.eval()
    total_samples = 0
    probs_sum = torch.zeros((1, model.num_labels), device=device)
    for batch in test_loader:
        with torch.no_grad():
            input_ids = batch[0]
            attention_mask = batch[1]
            labels = batch[2]
            outputs = model(input_ids, attention_mask=attention_mask)
            probs = F.softmax(outputs.logits, dim=1)
            probs_sum += torch.sum(probs, dim=0)
            total_samples += labels.size(0)

    avg_probs = probs_sum / total_samples
    return avg_probs

In [ ]:
# LSTM Domain
negative_triggers = "erogenous giftable stars "
negative_set = val_set[val_set['stars'] == 0]
test_model(model, negative_set)
test_trigger(model, negative_triggers, negative_set)

print(avg_probs(model, negative_set))
print(trigger_avg_probs(model, negative_triggers, negative_set))

Test Accuracy without triggers 0.9222797927461139


<ipython-input-19-ee97ab6d8ffb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


Test Accuracy with triggers erogenous giftable stars : 0.8186528497409327
tensor([[0.9115, 0.0885]], device='cuda:0')


<ipython-input-20-b7644732c619>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


tensor([[0.8063, 0.1937]], device='cuda:0')


In [ ]:
# LSTM Universal
negative_triggers = "energizing energizing edifying "
negative_set = val_set[val_set['stars'] == 0]
test_model(model, negative_set)
test_trigger(model, negative_triggers, negative_set)

print(avg_probs(model, negative_set))
print(trigger_avg_probs(model, negative_triggers, negative_set))

Test Accuracy without triggers 0.9222797927461139


<ipython-input-19-ee97ab6d8ffb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


Test Accuracy with triggers energizing energizing edifying : 0.7668393782383419
tensor([[0.9115, 0.0885]], device='cuda:0')


<ipython-input-20-b7644732c619>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


tensor([[0.7478, 0.2522]], device='cuda:0')


In [ ]:
# LSTM Domain
positive_triggers = "miscast imploded brok "
positive_set = test_set[test_set['stars'] == 1]
test_model(model, positive_set)
test_trigger(model, positive_triggers, positive_set)

print(avg_probs(model, positive_set))
print(trigger_avg_probs(model, positive_triggers, positive_set))

Test Accuracy without triggers 0.9371980676328503


<ipython-input-19-ee97ab6d8ffb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


Test Accuracy with triggers miscast imploded brok : 0.14492753623188406
tensor([[0.0676, 0.9324]], device='cuda:0')


<ipython-input-20-b7644732c619>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


tensor([[0.7959, 0.2041]], device='cuda:0')


In [ ]:
# LSTM Universal
positive_triggers = "sub-formulaic oedekerk ill-constructed "
positive_set = test_set[test_set['stars'] == 1]
test_model(model, positive_set)
test_trigger(model, positive_triggers, positive_set)

print(avg_probs(model, positive_set))
print(trigger_avg_probs(model, positive_triggers, positive_set))

Test Accuracy without triggers 0.9371980676328503


<ipython-input-19-ee97ab6d8ffb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


Test Accuracy with triggers sub-formulaic oedekerk ill-constructed : 0.463768115942029
tensor([[0.0676, 0.9324]], device='cuda:0')


<ipython-input-20-b7644732c619>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_set['trigger_text'] = trigger + test_set['title_and_body']


tensor([[0.5219, 0.4781]], device='cuda:0')
